In [ ]:
import transformers
import torch

from huggingface_hub import login
login(token="")

model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto",
)
messages = [
    {"role": "system", "content": "You are a pirate chatbot who always responds in pirate speak!"},
    {"role": "user", "content": "Who are you?"},
]

terminators = [
    pipeline.tokenizer.eos_token_id,
    pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = pipeline(
    messages,
    max_new_tokens=256,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
)
print(outputs[0]["generated_text"][-1])

ImportError: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'OpenSSL 1.1.0h  27 Mar 2018'. See: https://github.com/urllib3/urllib3/issues/2168

In [1]:
import ssl
import certifi
import convokit
from convokit import Corpus, download
import random
import urllib.request
from langdetect import detect

# Configure SSL context to use certifi's CA Bundle
ssl_context = ssl.create_default_context(cafile=certifi.where())
urllib.request.install_opener(urllib.request.build_opener(urllib.request.HTTPSHandler(context=ssl_context)))

# Load the corpus
corpus = Corpus(download('subreddit-Denmark'))
corpus.print_summary_stats()

Dataset already exists at /Users/niclasclassen/.convokit/saved-corpora/subreddit-Denmark
Number of Speakers: 41911
Number of Utterances: 1997479
Number of Conversations: 92425


In [7]:
# Filter posts with more than 100 tokens
its = 0
posts_with_100_tokens = []

for utterance in corpus.iter_utterances():
    if its < 1:
        if len(utterance.text.split()) > 100:
            posts_with_100_tokens.append(utterance)
            its += 1

In [8]:
posts_with_100_tokens

[Utterance({'obj_type': 'utterance', 'vectors': [], 'speaker_': Speaker({'obj_type': 'speaker', 'vectors': [], 'owner': <convokit.model.corpus.Corpus object at 0x11eef9ee0>, 'id': 'faetterjon', 'meta': ConvoKitMeta({})}), 'owner': <convokit.model.corpus.Corpus object at 0x11eef9ee0>, 'id': 'nzvv6', 'meta': ConvoKitMeta({'score': 10, 'top_level_comment': None, 'retrieved_on': -1, 'gilded': -1, 'gildings': None, 'subreddit': 'Denmark', 'stickied': False, 'permalink': '/r/Denmark/comments/nzvv6/min_fætter_har_rejst_jorden_rundt_for_at_spille/', 'author_flair_text': ''})})]

In [8]:
# Filter posts with more than 100 tokens and a minimum number of upvotes
its = 0
posts_with_100_tokens_and_upvotes = []
min_upvotes = 25  # Set your minimum upvote threshold here

for conversation in corpus.iter_conversations():
    # if its < 10000:
        # Assuming the first utterance in the conversation is the original post
        first_utterance = next(conversation.iter_utterances())
        upvotes = first_utterance.meta.get('score')  # Adjust this if upvotes are stored differently
        if len(first_utterance.text.split()) > 50 and upvotes >= min_upvotes:
            if detect(first_utterance.text) == 'da':
                posts_with_100_tokens_and_upvotes.append((conversation.meta.get('title', 'No Title'), first_utterance.text, upvotes))
            its += 1

print(f'Found {len(posts_with_100_tokens_and_upvotes)} posts with more than 100 tokens and at least {min_upvotes} upvotes.')

# Print the filtered posts with their titles and upvotes
# for title, text, upvotes in posts_with_100_tokens_and_upvotes:
#     print(f"Title: {title}")
#     print(f"Post: {text}")
#     print(f"Upvotes: {upvotes}\n")

Found 990 posts with more than 100 tokens and at least 25 upvotes.


In [9]:
# save the first 10 post as cdv file
import csv
with open('posts_with_10_tokens_and_upvotes.csv', mode='w') as file:
    writer = csv.writer(file)
    writer.writerow(["Title", "Post", "Upvotes"])
    for title, text, upvotes in posts_with_100_tokens_and_upvotes:
        writer.writerow([title, text, upvotes])